In [1]:
from math import pi
from math import cos

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import visdom
import numpy as np


In [2]:
cuda = torch.cuda.is_available()
batch_size = 64

# load data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(
                                    (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=True, download=True,
                     transform=transform),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=False, transform=transform),
    batch_size=batch_size, shuffle=True)


Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,
                               kernel_size=5,
                               stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.dense1 = nn.Linear(in_features=4 * 64, out_features=128)
        self.dense1_bn = nn.BatchNorm1d(128)
        self.dense2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_bn(self.conv3(x)), 2))
        x = x.view(-1, 4 * 64)
        x = F.relu(self.dense1_bn(self.dense1(x)))
        x = F.log_softmax(self.dense2(x))  # NLL loss expects log_softmax
        return x


In [4]:
def proposed_lr(initial_lr, iteration, epoch_per_cycle):
    # proposed learning late function
    return initial_lr * (cos(pi * iteration / epoch_per_cycle) + 1) / 2


In [5]:
def train_se(model, epochs, cycles, initial_lr, vis=None):
    """
    during an iteration a batch goes forward and backward  
    while during an epoch every batch of a data set is processed
    """
    snapshots = []
    _lr_list, _loss_list = [], []
    count = 0
    epochs_per_cycle = epochs // cycles
    optimizer = optim.SGD(model.parameters(), lr=initial_lr)

    for i in range(cycles):

        for j in range(epochs_per_cycle):
            _epoch_loss = 0

            lr = proposed_lr(initial_lr, j, epochs_per_cycle)
            optimizer.param_groups[0]['lr'] = lr

            for batch_idx, (data, target) in enumerate(train_loader):
                if cuda:
                    data, target = data.cuda(), target.cuda()

                optimizer.zero_grad()
                output = model(data)
                loss = F.nll_loss(output, target)
                _epoch_loss += loss.data/len(train_loader)
                loss.backward()
                optimizer.step()

            print(optimizer.param_groups[0]['lr'])
            _lr_list.append(optimizer.param_groups[0]['lr'])
            _loss_list.append(_epoch_loss.cpu())
            count += 1

            if vis is not None and j % 10 == 0:
                vis.line(np.array(_lr_list), np.arange(count), win="lr",
                         opts=dict(title="learning rate",
                                   xlabel="epochs",
                                   ylabel="learning rate (s.e.)"))
                vis.line(np.array(_loss_list), np.arange(count),  win="loss",
                         opts=dict(title="loss",
                                   xlabel="epochs",
                                   ylabel="training loss (s.e.)"))

        snapshots.append(model.state_dict())
    return snapshots


In [14]:
def train_se_loss(model, epochs, cycles, initial_lr, vis=None):
    """
    during an iteration a batch goes forward and backward  
    while during an epoch every batch of a data set is processed
    """
    snapshots = []
    loss_snapshots = []
    _lr_list, _loss_list = [], []
    count = 0
    epochs_per_cycle = epochs // cycles
    optimizer = optim.SGD(model.parameters(), lr=initial_lr)

    for i in range(cycles):

        for j in range(epochs_per_cycle):
            _epoch_loss = 0

            lr = proposed_lr(initial_lr, j, epochs_per_cycle)
            optimizer.param_groups[0]['lr'] = lr

            for batch_idx, (data, target) in enumerate(train_loader):
                if cuda:
                    data, target = data.cuda(), target.cuda()

                optimizer.zero_grad()
                output = model(data)
                loss = F.nll_loss(output, target)
                _epoch_loss += loss.data/len(train_loader)
                loss.backward()
                optimizer.step()

            _lr_list.append(lr)
            _loss_list.append(_epoch_loss.cpu())
            count += 1

            if vis is not None and j % 10 == 0:
                vis.line(np.array(_lr_list), np.arange(count), win="lr",
                         opts=dict(title="learning rate",
                                   xlabel="epochs",
                                   ylabel="learning rate (s.e.)"))
                vis.line(np.array(_loss_list), np.arange(count),  win="loss",
                         opts=dict(title="loss",
                                   xlabel="epochs",
                                   ylabel="training loss (s.e.)"))

        snapshots.append(model.state_dict())
        loss_snapshots.append(_loss_list[-1])
    return snapshots, loss_snapshots


In [7]:
def train_se_scheduler(model, epochs, cycles, initial_lr, vis=None):
    """
    during an iteration a batch goes forward and backward  
    while during an epoch every batch of a data set is processed
    """
    snapshots = []
    loss_snapshots = []
    _lr_list, _loss_list = [], []
    count = 0
    epochs_per_cycle = epochs // cycles
    optimizer = optim.SGD(model.parameters(), lr=initial_lr)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, epochs_per_cycle)
    for i in range(cycles):

        for j in range(epochs_per_cycle):
            _epoch_loss = 0

            for batch_idx, (data, target) in enumerate(train_loader):
                if cuda:
                    data, target = data.cuda(), target.cuda()

                optimizer.zero_grad()
                output = model(data)
                loss = F.nll_loss(output, target)
                _epoch_loss += loss.data/len(train_loader)
                loss.backward()
                optimizer.step()
            scheduler.step()

            _lr_list.append(scheduler.get_last_lr()[0])
            _loss_list.append(_epoch_loss.cpu())
            count += 1

            if vis is not None and j % 10 == 0:
                vis.line(np.array(_lr_list), np.arange(count), win="lr",
                         opts=dict(title="learning rate (scheduler)",
                                   xlabel="epochs",
                                   ylabel="learning rate (s.e.)"))
                vis.line(np.array(_loss_list), np.arange(count),  win="loss",
                         opts=dict(title="loss (scheduler)",
                                   xlabel="epochs",
                                   ylabel="training loss (s.e.)"))

        snapshots.append(model.state_dict())
        loss_snapshots.append(_loss_list[-1])
    return snapshots, loss_snapshots


In [8]:
def test_se(Model, snapshots, use_model_num):
    index = len(snapshots) - use_model_num
    snapshots = snapshots[index:]
    model_list = [Model() for _ in snapshots]

    for model, weight in zip(model_list, snapshots):
        model.load_state_dict(weight)
        model.eval()
        if cuda:
            model.cuda()

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        output_list = [model(data).unsqueeze(0) for model in model_list]
        output = torch.mean(torch.cat(output_list), 0).squeeze()
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

    return test_loss


In [15]:
def test_se_loss(Model, snapshots, loss_snapshots, use_model_num):
    # use the models with least losses
    loss_snapshots = np.array(loss_snapshots)
    index = np.argmin(loss_snapshots)
    snapshots = [snapshots[i] for i in index]
    model_list = [Model() for _ in snapshots]

    for model, weight in zip(model_list, snapshots):
        model.load_state_dict(weight)
        model.eval()
        if cuda:
            model.cuda()

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        output_list = [model(data).unsqueeze(0) for model in model_list]
        output = torch.mean(torch.cat(output_list), 0).squeeze()
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

    return test_loss


In [10]:
def train_normal(model, epochs, vis=None):

    optimizer = optim.Adam(model.parameters())
    _lr_list, _loss_list = [], []
    for epoch in range(epochs):
        _epoch_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            if cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            _epoch_loss += loss.data / len(train_loader)
            loss.backward()
            optimizer.step()

        _loss_list.append(_epoch_loss.cpu())
        _lr_list.append(optimizer.param_groups[0]['lr'])

        if vis is not None and epoch % 10 == 0:
            vis.line(np.array(_lr_list), np.arange(epoch+1), win="lr_n",
                     opts=dict(title="learning rate",
                               xlabel="epochs",
                               ylabel="learning rate (normal)"))
            vis.line(np.array(_loss_list), np.arange(epoch+1), win="loss_n",
                     opts=dict(title="loss",
                               xlabel="epochs",
                               ylabel="training loss (normal)"))

    return model


In [11]:
def test_normal(model):

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

    return test_loss


In [13]:
vis = visdom.Visdom(port=6006)
# model1, model2 = Net(), Net()
model1 = Net()
if cuda:
    model1.cuda()
    # model2.cuda()
print("snapshot ensemble")
models1, losses1 = train_se_loss(model1, 300, 6, 0.1, vis)
# models = train_se(model1, 300, 6, 0.1, vis)
# models, losses = train_se_scheduler(model1, 300, 6, 1, vis)
test_se(Net, models1, 5)
print("---")
print("snapshot ensemble (min loss)")
test_se_loss(Net, models1, losses1, 5)
# print("normal way")
# normal_model = train_normal(model2, 300, vis)
# test_normal(normal_model)


Setting up a new session...
C:\Users\jhcax\AppData\Local\Temp/ipykernel_36576/2159984635.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.dense2(x))  # NLL loss expects log_softmax


snapshot ensemble
0.1
0.09990133642141358
0.0996057350657239
0.09911436253643445
0.09842915805643156
0.09755282581475769
0.09648882429441258
0.09524135262330098
0.09381533400219318
0.09221639627510075
0.09045084971874738
0.08852566213878947
0.08644843137107058
0.08422735529643444
0.08187119948743449
0.07938926261462367
0.07679133974894983
0.07408768370508577
0.07128896457825364
0.06840622763423392
0.06545084971874737
0.062434494358242734
0.05936906572928624
0.05626666167821523
0.05313952597646568
0.05
0.04686047402353433
0.0437333383217848
0.04063093427071377
0.03756550564175726
0.03454915028125265
0.03159377236576612
0.02871103542174637
0.02591231629491423
0.02320866025105016
0.02061073738537635
0.018128800512565515
0.015772644703565566
0.013551568628929435
0.011474337861210543
0.009549150281252633
0.007783603724899258
0.006184665997806821
0.004758647376699033
0.0035111757055874327
0.0024471741852423235
0.0015708419435684518
0.000885637463565564
0.0003942649342761118
9.866357858642206

KeyboardInterrupt: 